In this notebook, I will:
* classify the water in classes from 0-4 according to the water classification scorecard
* export the water to a spreadsheet for dashboard creation.

In [1]:
import pandas as pd
import numpy as np

In [2]:
G_hydro_1 = pd.read_excel('y_LowerOIrangeEvW_OrigFile.xlsx','y_LowerOIrangeEvW_OrigFile')

In [3]:
# I will only use these following columns
# total number of columns: 20
df1 = G_hydro_1[[
                             'Monitoring Point Name',
                             'Latitude',
                             'Longitude',
                             'Drainage Region Name',
                             'Sample Start Date',
                             'pH-Diss-Water (PH) (pH units) Result',
                             'EC-Phys-Water (ELECTRICAL CONDUCTIVITY) (mS/m) Result',
                             'DMS-Tot-Water (DISSOLVED MAJOR SALTS) (mg/L) Result',
                             'Ca-Diss-Water (CALCIUM) (mg/L) Result',
                             'Mg-Diss-Water (MAGNESIUM) (mg/L) Result',
                             'Na-Diss-Water (SODIUM) (mg/L) Result',
                             'K-Diss-Water (POTASSIUM) (mg/L) Result',
                             'TAL-Diss-Water (TOTAL ALKALINITY AS CALCIUM CARBONATE) (mg/L) Result',
                             'Cl-Diss-Water (CHLORIDE) (mg/L) Result',
                             'SO4-Diss-Water (SULPHATE) (mg/L) Result',
                             'F-Diss-Water (FLUORIDE) (mg/L) Result',
                             'PO4-P-Diss-Water (ORTHO PHOSPHATE AS PHOSPHORUS) (mg/L) Result',
                             'Si-Diss-Water (SILICON) (mg/L) Result']]

In [4]:
# I create a new df with better naming
# 
df = pd.DataFrame()
df['Monitoring Point Name'] = df1['Monitoring Point Name']
df['Sample Start Date'] = df1['Sample Start Date']
df['Latitude']=df1['Latitude']
df['Longitude']=df1['Longitude']
df['Drainage Region Name'] = df1['Drainage Region Name']
df['DMS (mg/L)'] = df1['DMS-Tot-Water (DISSOLVED MAJOR SALTS) (mg/L) Result']
df['EC-(mS/m)'] = df1['EC-Phys-Water (ELECTRICAL CONDUCTIVITY) (mS/m) Result']
df['pH-Diss-Water (PH)'] = df1['pH-Diss-Water (PH) (pH units) Result']
df['Ca (mg/L)'] = df1['Ca-Diss-Water (CALCIUM) (mg/L) Result']
df['Mg (mg/L)'] = df1['Mg-Diss-Water (MAGNESIUM) (mg/L) Result']
df['Na (mg/L)']= df1['Na-Diss-Water (SODIUM) (mg/L) Result']
df['K (mg/L)'] = df1['K-Diss-Water (POTASSIUM) (mg/L) Result']
df['TAL CaCO3 (mg/L)'] = df1['TAL-Diss-Water (TOTAL ALKALINITY AS CALCIUM CARBONATE) (mg/L) Result']
df['Cl (mg/L)'] = df1['Cl-Diss-Water (CHLORIDE) (mg/L) Result']
df['SO4 (mg/L)'] = df1['SO4-Diss-Water (SULPHATE) (mg/L) Result']
df['F (mg/L)'] = df1['F-Diss-Water (FLUORIDE) (mg/L) Result']

In [5]:
# Define the mapping between drainage regions and resource units 23 GRU's
resource_units = {
    'GRU1': ['D71A','D71C','D71D','D72A','D72B','D73B','D72C','D73C','D73D','D73E','D73F'],
    
    'GRU2': ['D81A','D81B','D81D','D81E','D81F','D81G'],
    
    'GRU3': ['D82A','D82B','D82C','D82D',],
    
    'GRU4.1': ['D82E','D82F','D82G','D82H','D82J','D82K','D82L'],
    
    'GRU4.2': ['F10A','F10B','F10C','F20C','F20D','F20E'],
    
    'GRU6.1': ['D61A','D61B','D61C','D61D','D61E','D61F','D61G','D61H','D62C','D62D'],
    
    'GRU6.2': ['D61J','D61K','D61L','D61M','D62A','D62B','D62E','D62F'],
    
    'GRU6.3': ['D62H','D62G','D62J'],
    
    'GRU7.1': ['D51B','D51C','D52C','D55A','D55B','D55C','D55D','D55E','D55G','D55K','D56D','D56F','D56G','D56H','D56J'],
    
    'GRU7.2': ['D54A','D54B','D54C','D55F','D55H','D55J','D55L','D52D','D52E','D52F','D58A'],
    
    'GRU7.3': ['D53F','D54D','D54F','D57D'],
    
    'GRU7.4': ['D51A','D52A','D52B','D56A','D56B','D56C','D56E'],
    
    'GRU7.5': ['D54E','D55M','D57A','D57B','D57C','D58B','D58C'],
    
    'GRU7.6': ['D53D','D53G','D54G','D57E'],
    
    'GRU7.7': ['D53A','D53B','D53C','D53E','D53H','D53J'],
    
    'GRU8.1': ['F30A','F30B','F30C','F30D','F30E'],
    
    'GRU8.2': ['F40B','F40C','F40E','F40G','F50A','F50B','F50C','F50E','F50F','F20A','F20B','F20B','F20B','F30F','F30G'],
    
    'GRU8.3': ['F40A','F40D','F40F','F40H','F50G','F60A','F60B','F60C','F60D','F60E'],
    
    'GRU9.1': ['D41B','D41C','D41D','D41E','D41F','D41H','D41M'],
    
    'GRU9.2': ['D41G','D41L','D41J','D41K'],
    
    'GRU9.3': ['D71B','D73A'],
    
    'GRU10.1': ['D42A','D42B','D42C','D42D'],
    
    'GRU10.2': ['D42E','D81C']
}

# Create a new column 'resource_unit'
df['resource_unit'] = None

# Populate the 'resource_unit' column
def get_resource_unit(drainage_region):
    for ru, regions in resource_units.items():
        if drainage_region in regions:
            return ru
    return None

df['resource_unit'] = df['Drainage Region Name'].apply(get_resource_unit)

In [6]:
df = df.dropna()

In [7]:
import matplotlib.pyplot as plt

# Function to categorize according to ph
def categorize_pH(pH):
    if 5 <= pH < 9.5:
        return 'Class 0'
    elif 4.5 <= pH < 5:
        return 'Class 1'
    elif 9.5 <= pH < 10:
        return 'Class 1'
    elif 4 <= pH < 4.5:
        return 'Class 2'
    elif 10 <= pH < 10.5:
        return 'Class 2'
    elif 3 <= pH < 4:
        return 'Class 3'
    elif 10.5 <= pH < 11:
        return 'Class 3'
    elif pH >= 11:
        return 'Class 4'
    elif pH < 3:
        return 'Class 4'
# Create pH range column
df['pH_range'] = df['pH-Diss-Water (PH)'].apply(categorize_pH)

In [8]:
# Function to categorize according to DMS
def categorize_DMS(DMS):
    if DMS < 450:
        return 'Class 0'
    elif 450 <= DMS < 1000:
        return 'Class 1'
    elif 1000 <= DMS < 2400:
        return 'Class 2'
    elif 2400 <= DMS < 3400:
        return 'Class 3'
    elif DMS >= 3400:
        return 'Class 4'
# Create DMS range column
df['DMS_range'] = df['DMS (mg/L)'].apply(categorize_DMS)

In [9]:
# Function to categorize according to EC
def categorize_EC(EC):
    if EC < 70:
        return 'Class 0'
    elif 70 <= EC < 150:
        return 'Class 1'
    elif 150 <= EC < 370:
        return 'Class 2'
    elif 370 <= EC < 520:
        return 'Class 3'
    elif EC >= 520:
        return 'Class 4'
# Create EC range column
df['EC_range'] = df['EC-(mS/m)'].apply(categorize_EC)

In [10]:
# Function to categorize according to Ca
def categorize_Ca(Ca):
    if Ca < 80:
        return 'Class 0'
    elif 80 <= Ca < 150:
        return 'Class 1'
    elif 150 <= Ca < 300:
        return 'Class 2'
    elif Ca >= 300:
        return 'Class 3'
# Create Ca range column
df['Ca_range'] = df['Ca (mg/L)'].apply(categorize_Ca)

In [11]:
# Function to categorize according to Mg
def categorize_Mg(Mg):
    if Mg < 70:
        return 'Class 0'
    elif 70 <= Mg < 100:
        return 'Class 1'
    elif 100 <= Mg < 200:
        return 'Class 2'
    elif 200 <= Mg < 400:
        return 'Class 3'
    elif Mg >= 400:
        return 'Class 4'
# Create pH range column
df['Mg_range'] = df['Mg (mg/L)'].apply(categorize_Mg)

In [12]:
# Function to categorize according to Na
def categorize_Na(Na):
    if Na < 100:
        return 'Class 0'
    elif 100 <= Na < 200:
        return 'Class 1'
    elif 200 <= Na < 400:
        return 'Class 2'
    elif 400 <= Na < 1000:
        return 'Class 3'
    elif Na >= 1000:
        return 'Class 4'
# Create Na range column
df['Na_range'] = df['Na (mg/L)'].apply(categorize_Na)

In [13]:
# Function to categorize according to TAL CaCO3
def categorize_TAL(TAL):
    if TAL < 200:
        return 'Class 0'
    elif 200 <= TAL < 300:
        return 'Class 1'
    elif 300 <= TAL < 600:
        return 'Class 2'
    elif 600 <= TAL:
        return 'Class 3'
# Create TAL CaCO3 range column
df['TAL_range'] = df['TAL CaCO3 (mg/L)'].apply(categorize_TAL)

In [14]:
# Function to categorize according to Cl
def categorize_Cl(Cl):
    if Cl < 100:
        return 'Class 0'
    elif 100 <= Cl < 200:
        return 'Class 1'
    elif 200 <= Cl < 600:
        return 'Class 2'
    elif 600 <= Cl < 1200:
        return 'Class 3'
    elif Cl >= 1200:
        return 'Class 4'
# Create Cl range column
df['Cl_range'] = df['Cl (mg/L)'].apply(categorize_Cl)

In [15]:
# Function to categorize according to SO4
def categorize_SO4(SO4):
    if SO4 < 200:
        return 'Class 0'
    elif 200 <= SO4 < 400:
        return 'Class 1'
    elif 400 <= SO4 < 600:
        return 'Class 2'
    elif 600 <= SO4 < 1000:
        return 'Class 3'
    elif SO4 >= 1000:
        return 'Class 4'
# Create pH range column
df['SO4_range'] = df['SO4 (mg/L)'].apply(categorize_SO4)

In [16]:
# Function to categorize according to F
def categorize_F(F):
    if F < 0.7:
        return 'Class 0'
    elif 0.7 <= F < 1:
        return 'Class 1'
    elif 1 <= F < 1.5:
        return 'Class 2'
    elif 1.5 <= F < 3.5:
        return 'Class 3'
    elif F >= 3.5:
        return 'Class 4'
# Create pH range column
df['F_range'] = df['F (mg/L)'].apply(categorize_F)

In [17]:
# Create a new column 'water_class'
df['water_class'] = 0

# Apply classification rules based on the given criteria
conditions = [
    #Class 4
    (df['EC-(mS/m)']>= 520)        | 
    (df['pH-Diss-Water (PH)'] <3)   |
    (df['pH-Diss-Water (PH)'] >=11) |
    (df['Mg (mg/L)'] >=400)        | 
    (df['Na (mg/L)'] >=1000)        |
    (df['Cl (mg/L)'] >=1200)        |
    (df['SO4 (mg/L)'] >=1000)      | 
    (df['F (mg/L)'] >=3.5)   
    ,
    #Class 3
    ((df['EC-(mS/m)']>= 370) & (df['EC-(mS/m)'] <520))                 | 
    ((df['pH-Diss-Water (PH)']>=3) & (df['pH-Diss-Water (PH)'] <4))      |
    ((df['pH-Diss-Water (PH)']>=10.5) & (df['pH-Diss-Water (PH)'] <11))  |
    (df['Ca (mg/L)']>=300)                                              |
    ((df['Mg (mg/L)'] >=200) & (df['Mg (mg/L)'] <400))                 |
    ((df['Na (mg/L)'] >=400)& (df['Na (mg/L)'] <1000))                   |                             
    (df['TAL CaCO3 (mg/L)']>=600)                                       | 
    ((df['Cl (mg/L)'] >=600) & (df['Cl (mg/L)'] <1200))                  |
    ((df['SO4 (mg/L)'] >=600) & (df['SO4 (mg/L)'] <1000))              | 
    ((df['F (mg/L)'] >=1.5) & (df['F (mg/L)'] <3.5))    
    ,
    #Class 2
    ((df['EC-(mS/m)']>= 150) & (df['EC-(mS/m)'] <370))                | 
    ((df['pH-Diss-Water (PH)']>=4) & (df['pH-Diss-Water (PH)'] <4.5))   |
    ((df['pH-Diss-Water (PH)']>=10) & (df['pH-Diss-Water (PH)'] <10.5)) |
    ((df['Ca (mg/L)']>=150) & (df['Ca (mg/L)'] <300))                 |
    ((df['Mg (mg/L)'] >=100) & (df['Mg (mg/L)'] <200))                |
    ((df['Na (mg/L)'] >=200)& (df['Na (mg/L)'] <400))                   |                             
    ((df['TAL CaCO3 (mg/L)']>=300) & (df['TAL CaCO3 (mg/L)'] <600))   | 
    ((df['Cl (mg/L)'] >=200) & (df['Cl (mg/L)'] <600))                  |
    ((df['SO4 (mg/L)'] >=400) & (df['SO4 (mg/L)'] <600))              | 
    ((df['F (mg/L)'] >=1) & (df['F (mg/L)'] <1.5))
    ,
    #Class 1
    ((df['EC-(mS/m)']>= 70) & (df['EC-(mS/m)'] <150))                 | 
    ((df['pH-Diss-Water (PH)']>=4.5) & (df['pH-Diss-Water (PH)'] <5))   |
    ((df['pH-Diss-Water (PH)'] >=9.5) & (df['pH-Diss-Water (PH)'] <10)) |
    ((df['Ca (mg/L)']>=80) & (df['Ca (mg/L)'] <150))                  |
    ((df['Mg (mg/L)'] >=70) & (df['Mg (mg/L)'] <100))                 |
    ((df['Na (mg/L)'] >=100)& (df['Na (mg/L)'] <200))                   |                             
    ((df['TAL CaCO3 (mg/L)']>=200) & (df['TAL CaCO3 (mg/L)'] <300))   | 
    ((df['Cl (mg/L)'] >=100) & (df['Cl (mg/L)'] <200))                  |
    ((df['SO4 (mg/L)'] >=200) & (df['SO4 (mg/L)'] <400))              | 
    ((df['F (mg/L)'] >=0.7) & (df['F (mg/L)'] <1))
    ,
    #Class 0
    (df['EC-(mS/m)'] <70)         &
    ((df['pH-Diss-Water (PH)'] >5) & (df['pH-Diss-Water (PH)'] <9.5)) &
    (df['Ca (mg/L)'] <80)         &
    (df['Mg (mg/L)'] <70)         &
    (df['Na (mg/L)'] <100) &
    (df['TAL CaCO3 (mg/L)'] <200) &
    (df['Cl (mg/L)'] <100)         &
    (df['SO4 (mg/L)'] <200) & 
    (df['F (mg/L)'] <0.7)
]

choices = [4,3,2,1,0]


In [18]:
df['water_class'] = np.select(conditions, choices)

In [19]:
#print(df['water_class1'].value_counts())
print(df['water_class'].value_counts())

2    3179
4    2495
3    2351
1    1957
0     557
Name: water_class, dtype: int64


In [20]:
df_mqL = pd.DataFrame()
df_mqL['Monitoring Point Name'] = df['Monitoring Point Name']
df_mqL['Sample Start Date'] = df['Sample Start Date']
df_mqL['Latitude']=df['Latitude']
df_mqL['Longitude']=df['Longitude']
df_mqL['Drainage Region Name'] = df['Drainage Region Name']
df_mqL['resource_unit'] = df['resource_unit']
df_mqL['pH-Diss-Water (PH)'] = df['pH-Diss-Water (PH)']
df_mqL['DMS (mg/L)'] = df['DMS (mg/L)']
df_mqL['EC-(mS/m)'] = df['EC-(mS/m)']
df_mqL['Ca (mq/L)'] = df['Ca (mg/L)'] *0.0499
df_mqL['Mg (mq/L)'] = df['Mg (mg/L)'] *0.08224
df_mqL['Na (mq/L)']= df['Na (mg/L)'] *0.0435
df_mqL['K (mq/L)'] = df['K (mg/L)'] *0.02558
df_mqL['HCO3 (mq/L)'] = df1["TAL-Diss-Water (TOTAL ALKALINITY AS CALCIUM CARBONATE) (mg/L) Result"] * 0.01639 * 1.219
df_mqL['Cl (mq/L)'] = df['Cl (mg/L)'] * 0.0282
df_mqL['SO4 (mq/L)'] = df['SO4 (mg/L)']* 0.02082
df_mqL['F (mq/L)'] = df['F (mg/L)'] * 0.05263

In [21]:
# Export df to Excel
#df.to_excel('water_classification_mgL.xlsx', index=False)

In [22]:
# Export df_mqL to Excel
#df_mqL.to_excel('water_classification_meQ.xlsx', index=False)

In [23]:
#df['Usable_water'] = df['water_class'].apply(lambda x: 1 if x in [0, 1, 2] else 0)

In [24]:
df = df.drop(['pH_range', 'EC_range', 'Ca_range', 'Mg_range', 'Na_range', 'TAL_range', 'Cl_range', 'F_range',
              'SO4_range', 'Monitoring Point Name','Sample Start Date' ,'Latitude' ,
              'Longitude' ,'Drainage Region Name', 'resource_unit', 'DMS_range','DMS (mg/L)'], axis=1)

In [25]:
# A way to create a spreadsheet for average values per GRU
#region_averages = df.groupby('resource_unit').mean().round(2)

#print(region_averages)
# Save the results to a CSV file
#region_averages.to_csv("GRU_averages.csv")

In [26]:
df.head()

,EC-(mS/m),pH-Diss-Water (PH),Ca (mg/L),Mg (mg/L),Na (mg/L),K (mg/L),TAL CaCO3 (mg/L),Cl (mg/L),SO4 (mg/L),F (mg/L),water_class
0,40.0,8.42,29.8,12.2,28.6,2.53,120.9,25.3,29.6,0.22,0
1,37.3,8.27,29.1,11.3,23.5,2.56,122.8,19.0,35.1,0.24,0
2,36.9,8.18,27.6,10.6,25.0,2.77,108.8,16.8,29.2,0.25,0
3,38.3,8.16,30.8,11.4,24.1,2.81,112.7,17.7,39.7,0.23,0
4,35.2,8.18,28.6,10.6,20.8,2.27,100.0,14.2,35.2,0.21,0


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [28]:
X = df.drop('water_class', axis=1)
y = df['water_class']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
# Create different models
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100),
    "Support Vector Regression": SVR()
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Model: {name}")
    print(f"Mean Squared Error: {mse}")  
    
    print(f"R-squared: {r2}")
    print("-" * 30)

Model: Linear Regression
Mean Squared Error: 1.005895061202687
R-squared: 0.30108951037763354
------------------------------
Model: Random Forest
Mean Squared Error: 0.006336717267552182
R-squared: 0.9955971568617027
------------------------------
Model: Support Vector Regression
Mean Squared Error: 0.46077679366916396
R-squared: 0.6798455953398419
------------------------------


In [30]:

# Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(X_train, y_train)

# Feature importance
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature F (mg/L) (0.551222)
2. feature EC-(mS/m) (0.312590)
3. feature Na (mg/L) (0.065495)
4. feature TAL CaCO3 (mg/L) (0.032219)
5. feature Cl (mg/L) (0.028973)
6. feature SO4 (mg/L) (0.005570)
7. feature pH-Diss-Water (PH) (0.002964)
8. feature Ca (mg/L) (0.000529)
9. feature Mg (mg/L) (0.000228)
10. feature K (mg/L) (0.000211)


In [31]:
df = df.drop(['TAL CaCO3 (mg/L)', 'Cl (mg/L)',
              'SO4 (mg/L)', 'pH-Diss-Water (PH)',
             'Ca (mg/L)', 'K (mg/L)', 'Mg (mg/L)'], axis=1)

In [32]:
X = df.drop('water_class', axis=1)
y = df['water_class']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# Create different models
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100),
    "Support Vector Regression": SVR(),
}

print("Model prediction with only the top 3 features")
# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Model: {name}")
    print(f"Mean Squared Error: {mse}")  
    
    print(f"R-squared: {r2}")
    print("-" * 30)

Model prediction with only the top 3 features
Model: Linear Regression
Mean Squared Error: 1.1509508884567887
R-squared: 0.20030261603944421
------------------------------
Model: Random Forest
Mean Squared Error: 0.08408595825426944
R-squared: 0.9415758556527841
------------------------------
Model: Support Vector Regression
Mean Squared Error: 0.4748332068112927
R-squared: 0.6700789954524209
------------------------------
